# Necessary Imports

In [1]:
pip install transformers torch onnx openvino-dev[onnx] datasets accelerate openvino peft bitsandbytes trl -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 956.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 22.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Usi

In [2]:
!pip show transformers
!pip show accelerate

Name: transformers
Version: 4.42.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl
Name: accelerate
Version: 0.32.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl


In [3]:
!pip uninstall -y transformers accelerate
!pip install transformers[torch] accelerate

Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
Found existing installation: accelerate 0.32.1
Uninstalling accelerate-0.32.1:
  Successfully uninstalled accelerate-0.32.1
  Using cached transformers-4.42.4-py3-none-any.whl (9.3 MB)
  Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)


In [7]:
import pandas as pd
import torch
from datasets import Dataset
from random import randrange
#from peft import Loraconfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer

# Load and Save Pre-trained Model and Tokenizer

In [8]:
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m") #TinyLlama/TinyLlama-1.1B-Chat-v1.0, EleutherAI/gpt-neo-125m, Qwen/Qwen2-0.5B-Instruct
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")

model.save_pretrained("./my_gpt_model")
tokenizer.save_pretrained("./my_gpt_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

('./my_gpt_model/tokenizer_config.json',
 './my_gpt_model/special_tokens_map.json',
 './my_gpt_model/vocab.json',
 './my_gpt_model/merges.txt',
 './my_gpt_model/added_tokens.json',
 './my_gpt_model/tokenizer.json')

# Convert PyTorch Model to ONNX Format

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./my_gpt_model")
model = AutoModelForCausalLM.from_pretrained("./my_gpt_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "gptneo_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding=True)

print("Model converted to ONNX format successfully.")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/usr/local/lib/python3.10/dist-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:231: TracerWarning: torch.tensor results are registered as constants in the trace. You can safe

Model converted to ONNX format successfully.


# Convert ONNX Model to OpenVINO IR Format

In [ ]:
import openvino as ov
import os

ir_model = ov.convert_model("gptneo_model.onnx",
                            input=[ov.PartialShape([1, 4])])

ov.save_model(ir_model, "gptneo_ir_model.xml")

print("Model converted to IR format successfully.")

print("XML file exists:", os.path.exists("gptneo_ir_model.xml"))
print("BIN file exists:", os.path.exists("gptneo_ir_model.bin"))

Model converted to IR format successfully.
XML file exists: True
BIN file exists: True


# Pre Processing Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('/content/medquad.csv')

df = df[['question', 'answer']]

In [ ]:
train_val_df, sample_df = train_test_split(df, test_size=0.1, random_state=42) #using only sample df as the whole dataset is very big

In [ ]:
sample_df['text'] = 'Question:\n' + sample_df['question'] + '\n\nAnswer:\n' + sample_df['answer']
sample_df.drop(columns=['question','answer'], axis=1,inplace=True)
sample_df.to_csv('/content/pre_dataset.csv', index=False)

In [9]:
df = pd.read_csv('/content/pre_dataset.csv')

train = Dataset.from_pandas(df)
train_dataset_size = len(train)
print(train_dataset_size)

1642


In [10]:
train

Dataset({
    features: ['text'],
    num_rows: 1642
})

# Load and Prepare Model, Tokenizer for finetuning

In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback
from datasets import load_dataset, DatasetDict
import torch

model_name = "./my_gpt_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

train_dataset_size = len(train)
batch_size = 4
steps_per_epoch = train_dataset_size // batch_size

target_steps = 300
num_epochs = max(3, target_steps // steps_per_epoch)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_steps=100,
    save_total_limit=3,
)

class PrinterCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.is_local_process_zero and logs is not None:
            if 'loss' in logs:
                print(f"Step {state.global_step}: Training Loss: {logs['loss']:.5f}")
            else:
                print(f"Step {state.global_step}: Logs: {logs}")

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train,
    dataset_text_field='text',
    max_seq_length=1042,
    tokenizer=tokenizer,
    packing=True,
    callbacks=[PrinterCallback()]
)

trainer.train()

trainer.save_model("./fine_tuned_gpt_model")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:192: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will overri

Generating train split: 0 examples [00:00, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2858 > 2048). Running this sequence through the model will result in indexing errors


Step,Training Loss
10,2.179300
20,2.382100
30,2.279700
40,2.274400
50,2.251600
60,2.157800
70,2.179300
80,1.997300
90,2.105600
100,2.147900


Step 10: Training Loss: 2.17930
Step 20: Training Loss: 2.38210
Step 30: Training Loss: 2.27970
Step 40: Training Loss: 2.27440
Step 50: Training Loss: 2.25160
Step 60: Training Loss: 2.15780
Step 70: Training Loss: 2.17930
Step 80: Training Loss: 1.99730
Step 90: Training Loss: 2.10560
Step 100: Training Loss: 2.14790
Step 110: Training Loss: 2.00430
Step 120: Training Loss: 2.00270
Step 130: Training Loss: 1.90540
Step 140: Training Loss: 1.77060
Step 150: Training Loss: 1.90090
Step 160: Training Loss: 1.93340
Step 170: Training Loss: 1.95570
Step 180: Training Loss: 1.82630
Step 190: Training Loss: 1.74760
Step 200: Training Loss: 1.62550
Step 210: Training Loss: 1.89290
Step 220: Training Loss: 1.63590
Step 230: Training Loss: 1.77020
Step 240: Training Loss: 1.92290
Step 250: Training Loss: 1.75570
Step 260: Training Loss: 1.66520
Step 270: Training Loss: 1.65880
Step 280: Training Loss: 1.60050
Step 290: Training Loss: 1.74490
Step 300: Training Loss: 1.60360
Step 310: Training 

# Exporting the Fine-Tuned GPT-Neo Model to ONNX Format

In [13]:
import torch

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_model")
model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt_model")

model.eval()

class ORTModelForOnnxExport(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, input_ids, attention_mask):
        return self.model(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)[0]

ort_model = ORTModelForOnnxExport(model)

dummy_input = tokenizer("Hello, world!", return_tensors="pt")

dynamic_axes = {
    'input_ids': {0: 'batch_size', 1: 'sequence'},
    'attention_mask': {0: 'batch_size', 1: 'sequence'},
    'output': {0: 'batch_size', 1: 'sequence', 2: 'vocab_size'}
}

torch.onnx.export(ort_model,
                  (dummy_input.input_ids, dummy_input.attention_mask),
                  "fine_tuned_gptneo_model.onnx",
                  input_names=['input_ids', 'attention_mask'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes,
                  opset_version=11,
                  do_constant_folding=True)

print("Fine-tuned model converted to ONNX format successfully.")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:114: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/usr/local/lib/python3.10/dist-packages/transformers/modeling_attn_mask_utils.py:162: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/usr/local/lib/python3.10/dist-packages/transformers/models/gpt_neo/modeling_gpt_neo.py:231: TracerWarning: torch.tensor results are registered as constants in the trace. You can safe

Fine-tuned model converted to ONNX format successfully.


# Converting the Fine-Tuned GPT-Neo Model to OpenVINO IR Format

In [14]:
from openvino.tools import mo

ov_model = mo.convert_model("fine_tuned_gptneo_model.onnx",
                            input=[('input_ids', [1, -1]), ('attention_mask', [1, -1])],
                            output=['output'])

from openvino.runtime import serialize
serialize(ov_model, "fine_tuned_gptneo_ir_model.xml")

print("Fine-tuned model converted to IR format successfully.")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Fine-tuned model converted to IR format successfully.


# OpenVINO IR Format Fine Tuned Model Inference

In [36]:
import openvino.runtime as ov
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_model")

core = ov.Core()
model = core.read_model("fine_tuned_gptneo_ir_model.xml")
compiled_model = core.compile_model(model, "CPU")

infer_request = compiled_model.create_infer_request()

def generate_text(prompt, max_length=200): # The maximum length is 200; the answer may be short.
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.numpy()
    attention_mask = tokenizer(prompt, return_tensors="pt").attention_mask.numpy()

    for _ in range(max_length):
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask}

        results = infer_request.infer(inputs)
        logits = results[next(iter(results))]

        next_token_id = np.argmax(logits[0, -1, :])

        input_ids = np.concatenate([input_ids, np.array([[next_token_id]])], axis=1)
        attention_mask = np.concatenate([attention_mask, np.array([[1]])], axis=1)

        if next_token_id == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

prompt = "What is (are) Schwannomatosis ?"
generated_text = generate_text(prompt)
print("Generated text:", generated_text)

#True answer: Schwannomatosis is a rare form of neurofibromatosis that is primarily characterized by multiple schwannomas (benign tumors of the nervous system) in the absence of bilateral (affecting both sides) vestibular schwannomas. Signs and symptoms of the condition vary based on the size, location and number of schwannomas but may include pain; numbness; tingling; and/or weakness in the fingers and toes. Inherited forms of the disorder account for only 15 percent of all cases. In some of these families, schwannomatosis is caused by changes (mutations) in the SMARCB1 or LZTR1 genes; in other cases, the exact underlying cause is unknown. When inherited, the condition is passed down in an autosomal dominant manner with highly variable expressivity and reduced penetrance. Treatment is based on the signs and symptoms present in each person but may include medications and/or surgery.

Generated text: What is (are) Schwannomatosis?

Answer:
Schwannomatosis is a rare condition that affects the nervous system. It is a condition that affects the nervous system and is inherited in an autosomal recessive manner. The condition is caused by mutations in the SLC25A4 gene. The cause of the condition is unknown.


# Simple Fine Tuned Model Inference

In [48]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

fine_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt_model")
fine_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt_model")

In [31]:
fine_model.cuda() # Using GPU for generating text

prompt = "What are the symptoms of Crome syndrome ?"

inputs = fine_tokenizer(prompt, return_tensors='pt', truncation=True, padding=True)
input_ids = inputs.input_ids.cuda()
attention_mask = inputs.attention_mask.cuda()

outputs = fine_model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=200,
    temperature=0.6,
    pad_token_id=fine_tokenizer.eos_token_id
)

In [32]:
generated_text = fine_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)

What are the symptoms of Crome syndrome?

Answer:
What are the signs and symptoms of Crome syndrome? The Human Phenotype Ontology provides the following list of signs and symptoms for Crome syndrome. If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Abnormality of the skin - Abnormality of the nail - Abnormality of the palate - Abnormality of the face - Abnormality of the tongue - Abnormality of the hand - Abnormality of the foot - Abnormality of the face - Abnormality of the ribs - Abnormality of the face - Abnormality of the tongue - Abnormality of the tongue - Abnormality of the hand - Abnormality of the foot - Abnormality of the neck - Abnormality of the ribs -
